## Guilherme Michel Lima de Carvalho
### Implementação do classificador Naive-Bayes

In [37]:
import numpy as np
### Criando uma função para extrair os indices das observações de cada classe no conjunto de dados. 
def indices_data(y): ## Recebe os targets 
    n_classes = len(np.unique(y))
    classes_indices = []
    for i in range(n_classes): ## As classes devem ser ordenadas a partir de 0.
        classes_indices.append(np.where(y==i)[0].tolist()) ## Faz um append dos indices da classe i
    return classes_indices

In [38]:
def prob_classes(y,classes_indices):
    probs_class = []
    for i in range(len(classes_indices)):
        probs_class.append(len(classes_indices[i])/len(y))
    return probs_class

In [39]:
### Estimativa da média e variância para cada variável, condicionalmente a cada classe.
def estimativas(X,classes_indices):
    n_classes = len(classes_indices) ## Número de classes
    n_variaveis = X.shape[1]  ## quantidade de variáveis do conjunto
    estimativas_medias = np.zeros((n_classes,n_variaveis),dtype=np.float64) ## Para salvar as estimativas de médias p/ cada variável condicional a cada classe
    estimativas_var = np.zeros((n_classes,n_variaveis),dtype=np.float64) ## Para salvar as estimativas de variância p/ cada variável condicional a cada classe
    for i in range(n_classes):
        estimativas_medias[i,:] = X[classes_indices[i]].mean(axis=0) ## Calcula a média de cada variável condicional a classe i 
        estimativas_var[i,:] = X[classes_indices[i]].var(axis=0) ## Calcula a var de cada variável condicional a classe i 
    return estimativas_medias,estimativas_var

In [40]:
def gaussiana(media,var,x):
    return (1/np.sqrt(2*(np.pi)*var))*np.exp(-((x-media)**2)/(2*var))

In [41]:
def ajusta_naivebayes(X_treino,y_treino):
    n_classes = len(np.unique(y_treino)) ## Número de classes 
    classes_indices = indices_data(y_treino) ## Indices nos dados de cada classe
    probs_class = prob_classes(y_treino,classes_indices) ## Probs marginais estimativas
    media, var = estimativas(X_treino,classes_indices) ## Media e variancia estimadas no conjunto de treino
    return media,var,probs_class

In [42]:
def prediz_naivebayes(X_teste,media,var,probs_class):
    #### Usando as estimativas do conjunto treino para predizer a classe das observações do teste
    n_classes = media.shape[0]
    n_observacoes = len(X_teste) ## Número de observações no teste
    n_variaveis = X_teste.shape[1] ## Quantidade de covariáveis
    probabilidade_predita = np.zeros((n_observacoes,n_classes),dtype=np.float64) ### Probabilidade predita da observação ser de cada uma das classes

    for i in range(n_observacoes):
        for j in range(n_classes):
            probabilidade_predita[i][j] = np.sum(np.log(gaussiana(media[j],var[j],X_teste[i]))) + np.log(probs_class[j]) 
    
    return np.argmax(probabilidade_predita,axis=1)

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

cancer  = load_breast_cancer() 
X = cancer.data
y = cancer.target

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25,random_state=42)

media,var,prob_classes = ajusta_naivebayes(X_treino,y_treino)
y_pred = prediz_naivebayes(X_teste,media,var,prob_classes)

print('A acurácia do Naive-Bayes no conjunto de teste foi de ' + str(np.mean(y_pred == y_teste)))

A acurácia do Naive-Bayes no conjunto de teste foi de 0.951048951048951
